# 再帰型ニューラルネットワークを用いたテキスト分類

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(0)

## 訓練データの読み込みとモデルの学習

In [3]:
import torchtext
import MeCab
tagger = MeCab.Tagger('-Owakati')

In [4]:
def tokenizer(text):
    return tagger.parse(text)

TEXT = torchtext.data.Field(sequential=True, batch_first=True, tokenize=tokenizer, lower=True, fix_length=100)
LABEL = torchtext.data.Field(sequential=False)

In [5]:
# 自作のデータセットを用いる場合は，以下のコードを用いる．
# このコードでは，data/my_text_dataset フォルダに，タブ区切りで「テキスト」と「ラベル」を
# 1行に1組ずつ列挙した train.tsv および test.tsv のファイルがあることを想定している．
# 
# **注意**：以降のコードでは，バッチ（変数名を data とする）のテキストやラベルを data.text や data.label として
# アクセスしているが，TabularDataset クラスを用いる場合には，これを data.Text や data.Label と書き換えること．
# 2020年6月24日時点で，torchtextの中で変数名が一貫していないようである．
# 書き換えないと `'Batch' object has no attribute 'text'` というエラーが出る．

train, test = torchtext.data.TabularDataset.splits(path='.',
                                         train='train.tsv', test='test.tsv', format='tsv',
                                         fields=[('Text', TEXT), ('Label', LABEL)])

In [6]:
TEXT.build_vocab(train, max_size=25000)
# 事前学習済みの単語埋め込みベクトルを用いる場合は，以下のコードを用いる．
# TEXT.build_vocab(train, vectors="glove.6B.100d")

LABEL.build_vocab(train)

In [7]:
print(LABEL.vocab.stoi.items())

dict_items([('<unk>', 0), ('others', 1), ('sports', 2)])


In [8]:
print(list(TEXT.vocab.stoi.items())[:20])
print(len(TEXT.vocab))

[('<unk>', 0), ('<pad>', 1), (' ', 2), ('の', 3), ('、', 4), ('い', 5), ('と', 6), ('に', 7), ('ー', 8), ('で', 9), ('た', 10), ('て', 11), ('る', 12), ('し', 13), ('。', 14), ('な', 15), ('は', 16), ('が', 17), ('を', 18), ('ン', 19)]
2739


In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
trainloader, testloader = torchtext.data.BucketIterator.splits((train, test), batch_size=4, sort=False, device=device)

In [10]:
dataiter = iter(trainloader)

In [11]:
data = dataiter.__next__()
x, y = data.Text, data.Label
for x_i in x:
    print(' '.join(TEXT.vocab.itos[w] for w in x_i))
print([LABEL.vocab.itos[yi] for yi in y])

【   s p o r t s   w a t c h   】   ダ ル ビ ッ シ ュ   と   川 崎   宗   、   試 合   後   に   ツ イ ッ タ ー   で   激 励   北 海 道   日 本   ハ ム フ ァ イ タ ー ズ   の   エ ー ス ・ ダ ル ビ ッ シ ュ   有   と   、   福 岡   ソ フ ト バ ン ク ホ ー ク ス  
【   s p o r t s   w a t c h   】   興   毅   の   敗 戦   に   ジ ョ ー ジ   ＆   テ リ ー   は   「   勉 強   に   な っ   た   と   思 う   」   w b c   世 界   フ ラ イ   級   王 座   統 一   戦   で   は   、   暫 定   王 者   ポ ン サ ク レ ッ ク
気   を   つ け   て   ！   　   ス マ ー ト   フ ォ ン   、   海 外   パ ケ ッ ト   定 額   プ ラ ン   の   つ も り   が   約   1 0 0   万   円   ！   の   ナ ゾ   【   話 題   】   歩 行   者   に   あ ま り   出 会 わ   な い   街   、   し か も   言 葉  
【   s p o r t s   w a t c h   】   興   毅   が   成 長   ア ピ ー ル   「   1 0   段 階   で   言 う   た ら   ま だ   3   か   3   ,   5   」   プ ロ   ボ ク サ ー   ・   亀 田   興   毅   が   、   か ね て か ら   語 っ   て   い   た   “  
['sports', 'sports', 'others', 'sports']


In [12]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        output, (hn, cn) = self.lstm(x)
        hn = hn.squeeze(0)
        return self.fc(hn)

In [13]:
from itertools import islice
rnn = RNN(len(TEXT.vocab), 100, 30, 3)
# 事前学習済みの単語埋め込みベクトルを用いる場合は，以下のコードを挿入する．
# rnn.embedding.weight.data.copy_(TEXT.vocab.vectors)
rnn.to(device)
optimizer = optim.SGD(rnn.parameters(), lr = 0.01)
for epoch in range(10):
    sumloss = 0.0
    #for data in trainloader:  （計算資源が十分ある環境では，全てのデータを使う方が良い）
    for data in islice(trainloader, 250):
        x, y = data.Text, data.Label - 1
        optimizer.zero_grad()
        a = rnn(x)
        loss = F.cross_entropy(a, y)
        loss.backward()
        optimizer.step()
        sumloss += loss.item()
    print('epoch: {}, loss: {:.4f}'.format(epoch, sumloss))

epoch: 0, loss: 187.5182
epoch: 1, loss: 159.5307
epoch: 2, loss: 150.9242
epoch: 3, loss: 146.5221
epoch: 4, loss: 143.6548
epoch: 5, loss: 141.7055
epoch: 6, loss: 139.8902
epoch: 7, loss: 138.4223
epoch: 8, loss: 136.8523
epoch: 9, loss: 135.6241


In [14]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        x, y = data.Text, data.Label - 1
        a = rnn(x)
        pred_y = torch.argmax(a, dim=1)
        correct += (pred_y == y).sum().item()
        total += pred_y.size(0)

print(correct / total)

0.55


In [15]:
dataiter = iter(testloader)

In [16]:
data = dataiter.__next__()
x, y = data.Text, data.Label
for x_i in x:
    print(' '.join(TEXT.vocab.itos[w] for w in x_i))
a = rnn(x)
pred_y = torch.argmax(a, dim=1)
print([LABEL.vocab.itos[yi + 1] for yi in pred_y])
print([LABEL.vocab.itos[yi] for yi in y])

【   s p o r t s   w a t c h   】   ノ ム   さ ん   、   楽 天   新   監 督   に   「   な ん で   星 野   な   の   」   9   日   深 夜   、   t b s   の   ス ポ ー ツ   番 組   「   s   1   」   で   は   、   元   東 北   楽 天   ゴ ー ル デ ン
【   s p o r t s   w a t c h   】   闘 莉   王   、   ド イ ツ   w   杯   時   は   「   自 分   に   イ ラ イ ラ   し   て   た   」   日 本 テ レ ビ   「   n e w s   z e r o   」   （   2 3   日   放 送   分   ）   で   は   、   日 本   代
【   s p o r t s   w a t c h   】   韓 国   ・   全   北   サ ポ ー タ ー   が   卑 劣   な   横 断 幕   、   セ レ ッ ソ   は   悔 や ま   れ る   敗 戦   韓 国   の   全   州   で   行 わ   れ   た   、   ア ジ ア   ・   チ ャ ン ピ オ ン ズ   リ ー グ  
「   直 訳   か   よ   」   m l b   日 本   開 幕   戦   の   動 画   が   話 題   に   e s p n   ア メ リ カ   が   制 作   し   た   動 画   が   話 題   に   な っ   て   い る   。   動 画   は   、   m l b   日 本   開 幕   戦   「   マ リ ナ ー ズ
['others', 'others', 'others', 'sports']
['sports', 'sports', 'sports', 'sports']
